In [1]:
import numpy as np
import argparse
import sys
sys.path.insert(0, "../lib")
import utils
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import Formula
from rpy2.robjects import numpy2ri
numpy2ri.activate()

network = importr(
    'network',
    on_conflict="warn",
    robject_translations={
        'as.tibble.network': 'as_tibble_network',
        'as_tibble.network': 'as_tibble_network_'
    })

ergm = importr('ergm')
readRDS = robjects.r['readRDS']

In [ ]:
def compute_change_statistics(yt0, yt1, form_terms, diss_terms):
    """ Compute the change stat in mple for all edges  """
    # 1. take union.
    # 2. run ergmMPLE.
    # 3. adjust.

    ypos = np.logical_or(yt0, yt1).astype(int)
    yneg = np.logical_and(yt0, yt1).astype(int)
    nwpos = network.network(ypos)
    nwneg = network.network(yneg)

    fml_form = Formula('nwf ~ ' + ' + '.join(form_terms))
    fml_diss = Formula('nwd ~ ' + ' + '.join(diss_terms))
    fml_form.environment['nwf'] = nwpos
    fml_diss.environment['nwd'] = nwneg

    # compute change statistic with correction
    _lr_form = ergm.ergmMPLE(fml_form, output='array')
    lr_form_delta = np.array(_lr_form.rx('predictor')).squeeze(axis=0)
    _lr_diss = ergm.ergmMPLE(fml_diss, output='array')
    lr_diss_delta = np.array(_lr_diss.rx('predictor')).squeeze(axis=0)

    # correct and set diagonal to 0
    for k in range(len(form_terms)):
        lr_form_delta[:, :, k][yt0 == 1] = 0
        np.fill_diagonal(lr_form_delta[:, :, k], 0)
    for k in range(len(diss_terms)):
        lr_diss_delta[:, :, k][yt0 == 0] = 0
        np.fill_diagonal(lr_diss_delta[:, :, k], 0)

    return lr_form_delta, lr_diss_delta  # n x n x p1 (p2)


def save_H_y(data, conf, H_outfile, y_outfile):
    """Generate change statistics from temporal network data The script generates and save H.txt and y.txt"""
    data = np.array(data).astype(int)
    t = len(data)
    n = len(data[0])
    p = len(conf["form_terms"]) + len(conf["diss_terms"])

    print(f"time series length: {t}")
    print(f"network size: {n} x {n}")
    print(f"statistics dimension: {p}")

    H = np.zeros((t, n ** 2, p))  # compute H: T x E x p
    for i in range(t):
        if i == 0:
            lr_form, lr_diss = compute_change_statistics(
                yt0=data[i],
                yt1=data[i + 1],
                form_terms=conf["form_terms"],
                diss_terms=conf["diss_terms"])
        else:
            lr_form, lr_diss = compute_change_statistics(
                yt0=data[i - 1],
                yt1=data[i],
                form_terms=conf["form_terms"],
                diss_terms=conf["diss_terms"])
        H[i, :, :] = np.concatenate((lr_form, lr_diss), axis=2).reshape(n ** 2, p)  # row-first fill

    y = data.reshape(t, -1)

    print('Saving change statistics...')
    print(f"shape of H: {H.shape}")
    print(f"shape of y: {y.shape}")

    np.savetxt(H_outfile, H.reshape(H.shape[0], -1))
    np.savetxt(y_outfile, y)